In [46]:
import json
import spacy
from nlp_utils import *
import numpy as np

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

In [47]:
with open("data.json", "r") as f:
    data = json.load(f)["intents"]
    

- dictionary 


In [48]:
data_set = []

nlp = spacy.load("en_core_web_lg")

for intent in data:
    tag = intent["intent"]
    for pattern in intent["text"]:
        w = tokenize_to_vec(pattern, nlp)
        
        data_set.append((w, tag))



In [49]:
tag_translator = {}
count = 0

for (vec, tag) in data_set:
    if tag not in tag_translator.keys():
        tag_translator[tag] = count
        count += 1 

In [50]:
tag_translator

{'Greeting': 0,
 'GreetingResponse': 1,
 'CourtesyGreeting': 2,
 'CourtesyGreetingResponse': 3,
 'CurrentHumanQuery': 4,
 'NameQuery': 5,
 'RealNameQuery': 6,
 'TimeQuery': 7,
 'Thanks': 8,
 'NotTalking2U': 9,
 'UnderstandQuery': 10,
 'Shutup': 11,
 'Swearing': 12,
 'GoodBye': 13,
 'CourtesyGoodBye': 14,
 'WhoAmI': 15,
 'Clever': 16,
 'Gossip': 17,
 'Jokes': 18,
 'PodBayDoor': 19,
 'PodBayDoorResponse': 20,
 'SelfAware': 21}

In [51]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(data_set)
        self.X_data = []
        self.y_data = []
        
        for (words, tag) in data_set:
            self.X_data.append(words)
            self.y_data.append(tag_translator[tag])
        self.X_data = np.array(self.X_data)
        self.y_data = np.array(self.y_data)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
        

In [52]:
# hyperparameters
batch_size = 10
input_size = 300
hidden_size = 30
num_classes = 22
learning_rate = 0.01
num_epochs = 60

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda")
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) %10 == 0:
        print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.8f}")
        
              
print(f"final loss={loss.item():.8f}")      

epoch 10/60, loss=0.07633476
epoch 20/60, loss=0.00022511
epoch 30/60, loss=0.00000755
epoch 40/60, loss=0.10469776
epoch 50/60, loss=1.38555539
epoch 60/60, loss=0.00778239
final loss=0.00778239


In [53]:
save_data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_classes": num_classes,
    "tag_translator": tag_translator
}

torch.save(save_data, "model.save")

